In [1]:
import pandas as pd
import numpy as np

In [2]:
svm = pd.read_csv('it_ls_svm_all_features.csv')
bagging = pd.read_csv('ordered_predictions_bagging_svm_15_all_features.csv')

In [6]:
svm = svm.filter(items=['compound_name'],axis=1)

In [7]:
svm['rank_svm'] = svm.index

In [12]:
bagging = bagging.drop(['score_bagging'],axis=1)
#bagging : compound_name
#svm : 

In [13]:
list_svm = list(svm['compound_name'])
list_bagging = list(bagging['compound_name'])

In [17]:
list_common = []
common = 0
for i in list_svm:
    if i in list_bagging:
        list_common.append(i)

In [18]:
list_common

['Zr2AsC', 'Zr2GeC']